In [314]:
'''
M1. Integradora


Authors:
Luis Alberto Alcántara Cabrales A01634185
Alexa Serrano Negrete A01654063
Renet de Jesús Pérez Gómez A01640555
Vicente Javier Viera Guízar A01639784

Date:
11/17/2022
'''

#Model
import agentpy as ap
import random


#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython


In [315]:
parameters = {
    #Cantidad de carros
    'K': 20, 
    #Tamaño de ciudad
    'citySize': 60,
    #Duración simulación
    'steps': 100,
}

In [316]:
class CarAgent(ap.Agent):
    def setup(self): 
        '''
        0 = left 
        1 = right
        2 = top
        3 = botton
        '''       
        # self.idType = random.randint(0,3)
        self.idType = 2
        self.initialIdType = self.idType
        self.carChoseAWay = 0

class StreetAgent(ap.Agent):
    def setup(self):
       self.idType = 0

class TrafficLightAgent(ap.Agent):
    def setup(self):
        self.idType = 0
        self.timeCounterState = 0
        self.timeCounterWait = 0
        self.timeToStop = 4

    def determinateTime(self):
        # if self.idType == 0 or self.idType == 1:
        #     self.timeCounterState = 0
        #     self.timeCounterWait = 0
        if self.idType == 2 or self.idType == 3:
            self.timeCounterWait = self.timeToStop
            # self.timeCounterState = self.timeToStop

    def state(self):
        actualState = 2 # Go
        if self.timeCounterWait > 0:
            #Time to wait
            self.timeCounterWait -= 1
            actualState = 4 # Stop
        else:
            #Time to work             
            if self.timeCounterState == self.timeToStop - 1:
                actualState = 3 # Caution
            elif self.timeCounterState >= self.timeToStop:
                actualState = 4 # Danger
                self.timeCounterState = 0
                self.timeCounterWait = self.timeToStop
            self.timeCounterState += 1
        return actualState
        

In [317]:
def initialPositionStreetsAndAssignType(self):
    # Assign type of street
    aux = 0
    typeOfStreet = 0
    for street in self.streets:
        if aux <= self.endEnviroment:
            street.idType = typeOfStreet
            aux += 1
        else:
            typeOfStreet += 1
            street.idType = typeOfStreet
            aux = 1

    # Get initial position
    position = []
    
    for i in range(self.endEnviroment):
        position.append((self.leftSideStreet, i))    
    position.append((self.leftSideStreet, self.endEnviroment))    

    for i in range(self.endEnviroment):
        position.append((self.rightSideStreet, i))
    position.append((self.rightSideStreet, self.endEnviroment))        
    
    for i in range(self.endEnviroment):
        position.append((i, self.rightSideStreet))
    position.append((self.endEnviroment, self.rightSideStreet))
    
    for i in range(self.endEnviroment):
        position.append((i, self.leftSideStreet))
    position.append((self.endEnviroment, self.leftSideStreet))
    
    return position

In [318]:
class MyModel(ap.Model):
    
    def setup(self):
        # Find special coordinates
        
        # Fundamental coordinates
        self.leftSideStreet = int(self.p.citySize / 2)
        self.rightSideStreet = self.leftSideStreet + 1
        self.endEnviroment = self.p.citySize - 1

        # Find the coordinates of center 
        self.centerCoordinates = [
            [self.leftSideStreet, self.leftSideStreet],
            [self.leftSideStreet, self.rightSideStreet],
            [self.rightSideStreet, self.leftSideStreet],
            [self.rightSideStreet, self.rightSideStreet],
        ]

        # Find crosswalk
        self.crosswalkCoordinates = [
            [self.rightSideStreet, self.leftSideStreet - 2],
            [self.leftSideStreet, self.rightSideStreet + 2],
            [self.leftSideStreet - 2, self.leftSideStreet],
            [self.rightSideStreet + 2, self.rightSideStreet],
        ]

        # Find initial positions
        self.initialPositionCoordinates = [
            [self.rightSideStreet, 0],
            [self.leftSideStreet, self.endEnviroment],
            [0, self.leftSideStreet],
            [self.endEnviroment, self.rightSideStreet]
        ]

        # Find final positions
        self.finalPositionCoordinates = [
            [self.rightSideStreet, self.endEnviroment],
            [self.leftSideStreet, 0],
            [self.endEnviroment, self.leftSideStreet],
            [0, self.rightSideStreet]
        ]
        
        # Store type of movement
        self.move = [
            [0, 1], [0, -1], [1, 0], [-1, 0],
            [-1, 1], [1, -1], [1, 1], [-1, -1]
        ]

        # Create agents
        self.cars = ap.AgentList(self, self.p.K, CarAgent)
        self.streets = ap.AgentList(self, int(self.p.citySize * 4), StreetAgent)
        self.trafficLights = ap.AgentList(self, 4, TrafficLightAgent)
        

        # Create enviroment
        self.area = ap.Grid(self, [self.p.citySize] * 2)


        # Add agents to the enviroment
        #Cars
        positionCars = []
        for car in self.cars:
            positionCars.append(self.initialPositionCoordinates[car.idType])
            
        self.area.add_agents(self.cars, positionCars)
        
        # Street
        positionStreet = initialPositionStreetsAndAssignType(self)
        self.area.add_agents(self.streets, positionStreet)
        
        for street in self.streets:
            print("ID street: ", street.id, " - type street: ", street.idType, ", position: ", self.area.positions[street])

        #trafficLights - fix the positions
        '''
            The position in the grid of the traffic light doesn't import 
            it's more about decoration. The position in the "array" 
            is what really really
        '''
        positionTrafficLight = [
            [self.crosswalkCoordinates[3][0] - 1, self.crosswalkCoordinates[3][1] + 2], # Traffic light for street 0
            [self.crosswalkCoordinates[2][0] + 1, self.crosswalkCoordinates[2][1] - 2], # Traffic light for street 1
            [self.crosswalkCoordinates[0][0] + 2, self.crosswalkCoordinates[0][1] + 1], # Traffic light for street 2
            [self.crosswalkCoordinates[1][0] - 2, self.crosswalkCoordinates[1][1] - 1], # Traffic light for street 3
        ]

        self.area.add_agents(self.trafficLights, positionTrafficLight)
        
        #Assign the type of traffic light and its time to works
        counterTrafficLight = 0
        for trafficLight in self.trafficLights:
            trafficLight.idType = counterTrafficLight
            trafficLight.determinateTime()
            counterTrafficLight += 1


        # Define type of agent
        self.cars.agent_type = 0
        self.streets.agent_type = 1

        for trafficLight in self.trafficLights:
            if trafficLight.idType == 0 or trafficLight.idType == 1:
                trafficLight.agent_type = 2 # That type of traffic light start in "Green"
            elif trafficLight.idType == 2 or trafficLight.idType == 3:
                trafficLight.agent_type = 4 # That type of traffic light start in "Red"

        
        #Test
        # self.cross = ap.AgentList(self, 4, StreetAgent)
        # self.area.add_agents(self.cross, self.crosswalkCoordinates)
        #self.cross.agent_type = 3


            

    def step(self):
        for car in self.cars:
            #A simple movement
            for neighbor in self.area.neighbors(car):
                # print("agent: ", car.id, " - agent_type: ", neighbor.agent_type, " - Type: ", car.idType, " = ", neighbor.idType)
                if (neighbor.agent_type == 1 and car.idType == neighbor.idType):
                    # print("Go ahead - agent: ", car.id)
                    self.area.move_by(car, self.move[car.idType])
                    break
                elif neighbor.agent_type == 0:
                    # print("Detect a car, idCar: ", neighbor.id)
                    # Verify if the neighbor is really in front of our current car
                    [yCar, xCar] = self.area.positions[car]
                    [yNeighbor, xNeighbor] = self.area.positions[neighbor]
                    posY = yNeighbor - yCar
                    posX = xNeighbor - xCar

                    #Movement in that street
                    [movY, movX] = self.move[car.idType]
                    if (posY == movY and movX == posX):
                        # print("The car is behind of another car")
                        # print("Position of the neighbor: ", self.area.positions[neighbor])
                        # print("Position of our current car: ", self.area.positions[car])
                        break                
                    
            # The current car reach the crosswalk and choose a way        
            # [y, x] = self.area.positions[car]
            
            # if (self.crosswalkCoordinates[car.initialIdType][0] == y
            #     and self.crosswalkCoordinates[car.initialIdType][1] == x
            #     and car.carChoseAWay == 0):
            #         if car.idType == 0:
            #             car.idType = random.choice([0, 2, 3])

            #         if car.idType == 1:
            #             car.idType = random.choice([1, 2, 3])

            #         if car.idType == 2:
            #             car.idType = random.choice([0, 1, 2])

            #         if car.idType == 3:
            #             car.idType = random.choice([0, 1, 3])
    
            #         car.carChoseAWay = 1 # This attribute helps us prevent the car from choose a way again
            #         print("The car with the id ", car.id, "- choose: ", car.idType)
            # print("Final of the iteration of the car")

            # Verify if the car reached the destination coordinates 
            [currentY, currentX] = self.area.positions[car]
            [destinationY, destinationX] = self.finalPositionCoordinates[car.idType]
            if currentY == destinationY and destinationX == currentX:
                car.agent_type = 1

        # Update state of the traffic lights
        for trafficLight in self.trafficLights:
            trafficLight.agent_type = trafficLight.state()


In [319]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.area.attr_grid('agent_type')
    color_dict = {0:'#c1c1c1', 1: '#007', 2: '#3ECC06', 3: '#F7F30E', 4: '#FF5733', None:'#FFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic Simulation\n"
                 f"Time-step: {model.t}",
                 color="white") 

fig, ax = plt.subplots()
model = MyModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))

ID street:  21  - type street:  0 , position:  (30, 0)
ID street:  22  - type street:  0 , position:  (30, 1)
ID street:  23  - type street:  0 , position:  (30, 2)
ID street:  24  - type street:  0 , position:  (30, 3)
ID street:  25  - type street:  0 , position:  (30, 4)
ID street:  26  - type street:  0 , position:  (30, 5)
ID street:  27  - type street:  0 , position:  (30, 6)
ID street:  28  - type street:  0 , position:  (30, 7)
ID street:  29  - type street:  0 , position:  (30, 8)
ID street:  30  - type street:  0 , position:  (30, 9)
ID street:  31  - type street:  0 , position:  (30, 10)
ID street:  32  - type street:  0 , position:  (30, 11)
ID street:  33  - type street:  0 , position:  (30, 12)
ID street:  34  - type street:  0 , position:  (30, 13)
ID street:  35  - type street:  0 , position:  (30, 14)
ID street:  36  - type street:  0 , position:  (30, 15)
ID street:  37  - type street:  0 , position:  (30, 16)
ID street:  38  - type street:  0 , position:  (30, 17)
ID